<a href="https://colab.research.google.com/github/Youk520/profile/blob/master/%E5%A5%BD%E5%A5%BD%E5%AD%A6%E4%B9%A0nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import copy
import torch
from torch import nn
from torch import optim
import torchtext
from torchtext import data
from torchtext import datasets

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "ningye" # username from the json file
os.environ['KAGGLE_KEY'] = "ebaa4dbd87f76fd6edd6c247ca44df98" # key from the json file
!kaggle competitions download -c hecmontrealdeeplearningcourse # api copied from kaggle

if not os.path.exists('./text.csv'):
  !unzip text.csv.zip

  0% 0.00/561k [00:00<?, ?B/s]
100% 561k/561k [00:00<00:00, 38.2MB/s]
  0% 0.00/69.5k [00:00<?, ?B/s]
100% 69.5k/69.5k [00:00<00:00, 68.3MB/s]
  0% 0.00/94.4k [00:00<?, ?B/s]
100% 94.4k/94.4k [00:00<00:00, 84.5MB/s]
  0% 0.00/94.5k [00:00<?, ?B/s]
100% 94.5k/94.5k [00:00<00:00, 96.2MB/s]
  0% 0.00/798k [00:00<?, ?B/s]
100% 798k/798k [00:00<00:00, 94.8MB/s]
Archive:  text.csv.zip
  inflating: text.csv                


In [0]:
import pandas as pd   ##第一步先把id的数字转化成sequence

text_csv = pd.read_csv('text.csv')
train_csv = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')

print(train_csv)

title = text_csv.title.values
title_id =  text_csv.id.values
train_id = train_csv.id.values
train_label = train_csv.label.values
test_id = test_csv.id.values

train_title = title[train_id]
test_title = title[test_id]

print(train_id)

##print(len(train_title))
##print(len(test_title))


          id  label
0          0      1
1          3      1
2          6      1
3          8      0
4          9      0
...      ...    ...
12774  25547      4
12775  25548      3
12776  25554      2
12777  25555      2
12778  25557      1

[12779 rows x 2 columns]
[    0     3     6 ... 25554 25555 25557]


In [0]:
import numpy as np
data_array = np.concatenate((title[train_id].reshape(-1,1),train_label.reshape(-1,1)),axis = 1)  ##把两列合在一起变成一个矩阵 seq与对应的label
print(data_array.shape)
print(data_array)

data_index = np.arange(data_array.shape[0])
print(data_index)
##打乱顺序
np.random.shuffle(data_index)

print(data_index)
##打乱顺序分数据集
valid_index = data_index[:int(0.2*data_array.shape[0])]  ##分数据集 变成整形0.2validation

train_index = data_index[int(0.8*data_array.shape[0]):] ##train 0.8

print(valid_index)
print(train_index)

##把np array改成更csv
np.savetxt('train_titles.csv', data_array[train_index], delimiter=',', fmt=['%s' , '%d'],header='title,label',comments='')
np.savetxt('valid_titles.csv', data_array[valid_index], delimiter=',', fmt=['%s' , '%d'],header='title,label',comments='')
train_csv = pd.read_csv('train_titles.csv')
print(train_csv)

np.savetxt('test_titles.csv', title[title_id], delimiter=',', fmt='%s')  ##第一列跟第二列是什么形式 title and label
test_titles = pd.read_csv('test_titles.csv',header=None) ####test的title保存起来
print(test_titles)



(12779, 2)
[['interactive visual exploration of neighbor based patterns in data streams'
  1]
 ['relational division four algorithms and their performance' 1]
 ['simplifying xml schema effortless handling of nondeterministic regular expressions'
  1]
 ...
 ['an analysis of transformational analogy general framework and complexity'
  2]
 ['exploiting known taxonomies in learning overlapping concepts' 2]
 ['maintaining materialized views in distributed databases' 1]]
[    0     1     2 ... 12776 12777 12778]
[ 1514 11187  7586 ... 10976  8250  5215]
[ 1514 11187  7586 ...  5992 11875  9496]
[ 2783 11491   401 ... 10976  8250  5215]
                                                  title  label
0     deriving non redundant approximate association...      0
1     enhancing cross language information retrieval...      0
2                       classification enhanced ranking      0
3     determining cylindrical shape from contour and...      2
4           learning to connect language and pe

In [0]:
from torchtext.data import Field
from torchtext import data


tokenize = lambda x: x.split()  ##把seq出现空格的就打散
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = Field(sequential=False, use_vocab=False)   ##nlp预备知识

### load data splits
train_dataset, valid_dataset = data.TabularDataset.splits(
    path = os.getcwd(), format = 'csv', skip_header = True,
    train='train_titles.csv', validation='valid_titles.csv',   ##函数需要csv格式
    fields=[
        ('title', TEXT),   ##title seq
        ('label', LABEL)   
    ]
)

# build dictionary
TEXT.build_vocab(train_dataset)
LABEL.build_vocab(train_dataset)


vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 128
hidden_dim = 128


# build iterators        不需要用到text iterator
train_iter, val_iter = data.BucketIterator.splits(
    (train_dataset, valid_dataset), 
    batch_size=32)   ##batch size 都可以改


#for batch in train_iter:     ##iterater把seq 变成了vector 这事可以开始训练模型
 # print(batch.title)      ##不用print 这里只需要看效果


In [0]:
##如何衔接起来 assignment1 做iterator
def train(model, iterator, optimizer, criterion):
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        logits = model(batch.text.cuda())
        predictions = torch.max(logits, dim=-1)[1]
        loss = criterion(logits, batch.label.cuda())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += torch.eq(predictions, batch.label.cuda()).sum().item()
        total_prediction += batch.label.size(0)
    return total_loss / len(iterator), total_correct / total_prediction

In [0]:
# evaluate a model
# DO NOT MODIFY
def evaluate(model, iterator, criterion):  
    total_loss, total_correct, total_prediction = 0.0, 0.0, 0.0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            logits = model(batch.text.cuda())
            predictions = torch.max(logits, dim=-1)[1]
            loss = criterion(logits, batch.label.cuda())

            total_loss += loss.item()
            total_correct += torch.eq(predictions, batch.label.cuda()).sum().item()
            total_prediction += batch.label.size(0)
    return total_loss / len(iterator), total_correct / total_prediction

In [0]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, padding_idx):
        super(RNNClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        self.num_layers = 1
        # add the layers required for sentiment analysis.
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, self.num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, label_size)

    def zero_state(self, batch_size): 
        # implement the function, which returns an initial hidden state.
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        Init_state = (torch.zeros(self.num_layers, batch_size, self.hidden_dim).cuda(),torch.zeros(self.num_layers, batch_size, self.hidden_dim).cuda()) 

        return Init_state

    def forward(self, text):
        # implement the forward function of the model.
        hidden = self.zero_state(text.size(0))
        embedding = self.embedding(text)
        lstm_out, (lstm_h, lstm_c)= self.lstm(embedding, hidden)
        lstm_h = F.relu(torch.squeeze(lstm_h))
        lstm_h = self.fc1(lstm_h)


        return lstm_h

In [0]:
model = RNNClassifier(vocab_size, embedding_dim, hidden_dim, label_size, padding_idx)
# tune the optimizer type and hyperparameters here
optimizer = optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
model.cuda()
criterion.cuda()

# train and test the model
# DO NOT MODIFY
best_valid_acc = 0.0
best_state_dict = copy.deepcopy(model.state_dict())
for epoch in range(20):
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)

    print('Epoch {} | Train loss {:.3f} | Valid loss {:.3f} | Valid acc {:.3f}'.format(epoch, train_loss, valid_loss, valid_acc))

    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_state_dict = copy.deepcopy(model.state_dict())

AttributeError: ignored